## A notebook to seamlessly take blast output to GO Slim list

This is a notebook meant to run in a working directory. Please set working directory as variable in next cell

In [1]:
wd="/Volumes/Serine/wd/19-02-25"


In [2]:
cd $wd

/Volumes/Serine/wd/19-02-25


In [14]:
!curl -O https://raw.githubusercontent.com/hputnam/EastOyEpi/master/analyses/filtered.Cv_gene_sprot.05.blastout


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1984k  100 1984k    0     0  20.2M      0 --:--:-- --:--:-- --:--:-- 20.3M


In [4]:
!curl -O http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/uniprot-SP-GO.sorted

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  340M  100  340M    0     0  15.0M      0  0:00:22  0:00:22 --:--:-- 8509k0  0:00:16  0:00:09  0:00:07 28.9M     0  20.7M      0  0:00:16  0:00:10  0:00:06 26.0M


In [5]:
!curl -O http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/GO-GOslim.sorted

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2314k  100 2314k    0     0  3165k      0 --:--:-- --:--:-- --:--:-- 3165k


In [72]:
!curl -O http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/GO-GOslim.sorted

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2314k  100 2314k    0     0   874k      0  0:00:02  0:00:02 --:--:--  874k


### In this directory you will need three files
1) blastout file in format `-6`    
2) Uniprot GO annotation file (340M) available here `http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/uniprot-SP-GO.sorted`    
3) GOslim file available here `http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/GO-GOslim.sorted`

In [73]:
#checking if files in directory
!ls 

Blastquery-GOslim.tab              _blast-sort.tab
GO-GOslim.sorted                   _intermediate.file
_blast-GO-unfolded.sorted          filtered.Cv_gene_sprot.05.blastout
_blast-GO-unfolded.tab             uniprot-SP-GO.sorted
_blast-annot.tab                   {}.fasta2tab
_blast-sep.tab


### Set blastout file as variable

In [16]:
blastout="filtered.Cv_gene_sprot.05.blastout"


# That should be the last thing you have to Type!

![Run all cell below](https://d1zjcuqflbd5k.cloudfront.net/files/acc_524195/1CsBk?response-content-disposition=inline;%20filename=Screen%20Capture%20on%202017-07-20%20at%2011-50-11.gif&Expires=1500576917&Signature=gyehwKgCTxTHzRrBID1ecQc-aBTBD0si7-XVhQHvbDdboJ2j86KMTwN8XoOZzue7PPY-QT7tIDG4DGTeWXv73VT6zfylPqDz9u19oiGZOyiyyK6EAGlK~8nPzTvwhtRN1uNcqqBuINV94sb2HjK9c6TgsHf1N5eRZF4~4FhIgjY_&Key-Pair-Id=APKAJTEIOJM3LSMN33SA)

In [17]:
!head -2 $blastout

NC_035780.1:10072460-10102370	sp|Q6GQT9|NOMO1_MOUSE	72.340	47	13	0	14798	14658	46	92	8.13e-15	87.0
NC_035780.1:10136213-10167580	sp|Q7ZTV5|FSHR_CAIMO	43.421	152	76	2	29666	30091	529	680	6.61e-27	125


In [18]:
#convert pipes to tab
!tr '|' '\t' < $blastout \
> _blast-sep.tab

In [19]:
!head -2 _blast-sep.tab

NC_035780.1:10072460-10102370	sp	Q6GQT9	NOMO1_MOUSE	72.340	47	13	0	14798	14658	46	92	8.13e-15	87.0
NC_035780.1:10136213-10167580	sp	Q7ZTV5	FSHR_CAIMO	43.421	152	76	2	29666	30091	529	680	6.61e-27	125


In [22]:
#reducing number of columns and sorting 
!awk -v OFS='\t' '{print $3, $1, $13}' < _blast-sep.tab | sort \
> _blast-sort.tab

In [23]:
!head -2 _blast-sort.tab

A0A0A7DNP6	NC_035781.1:20079207-20081370	4.77e-06
A0A0B4J1F4	NC_035782.1:29450394-29476971	3.00e-20


In [24]:
#joining blast with uniprot annoation file and reducing to three columns UniprotID, Query, All GO terms
!join -t $'\t' \
_blast-sort.tab \
uniprot-SP-GO.sorted \
| cut -f1,2,14 \
> _blast-annot.tab

In [25]:
!head -2 _blast-annot.tab

A0A0A7DNP6	NC_035781.1:20079207-20081370	GO:0005576; GO:0007218
A0A0B4J1F4	NC_035782.1:29450394-29476971	GO:0005768; GO:0005769; GO:0005886; GO:0051443


## The following is a script modidified from Sam

In [26]:
%%bash 

# This script was originally written to address a specific problem that Rhonda was having



# input_file is the initial, "problem" file
# file is an intermediate file that most of the program works upon
# output_file is the final file produced by the script
input_file="_blast-annot.tab"
file="_intermediate.file"
output_file="_blast-GO-unfolded.tab"

# sed command substitutes the "; " sequence to a tab and writes the new format to a new file.
# This character sequence is how the GO terms are delimited in their field.
sed $'s/; /\t/g' "$input_file" > "$file"

# Identify first field containing a GO term.
# Search file with grep for "GO:" and pipe to awk.
# Awk sets tab as field delimiter (-F'\t'), runs a for loop that looks for "GO:" (~/GO:/), and then prints the field number).
# Awk results are piped to sort, which sorts unique by number (-ug).
# Sort results are piped to head to retrieve the lowest value (i.e. the top of the list; "-n1").
begin_goterms=$(grep "GO:" "$file" | awk -F'\t' '{for (i=1;i<=NF;i++) if($i ~/GO:/) print i}' | sort -ug | head -n1)

# While loop to process each line of the input file.
while read -r line
	do
	
	# Send contents of the current line to awk.
	# Set the field separator as a tab (-F'\t') and print the number of fields in that line.
	# Save the results of the echo/awk pipe (i.e. number of fields) to the variable "max_field".
	max_field=$(echo "$line" | awk -F'\t' '{print NF}')

	# Send contents of current line to cut.
	# Cut fields (i.e. retain those fields) 1-12.
	# Save the results of the echo/cut pipe (i.e. fields 1-12) to the variable "fixed_fields"
	fixed_fields=$(echo "$line" | cut -f1-2)

	# Since not all the lines contain the same number of fields (e.g. may not have GO terms),
	# evaluate the number of fields in each line to determine how to handle current line.

	# If the value in max_field is less than the field number where the GO terms begin,
	# then just print the current line (%s) followed by a newline (\n).
	if (( "$max_field" < "$begin_goterms" ))
		then printf "%s\n" "$line"
			else

			# Send contents of current line (which contains GO terms) to cut.
			# Cut fields (i.e. retain those fields) 13 to whatever the last field is in the curent line.
			# Save the results of the echo/cut pipe (i.e. all the GO terms fields) to the variable "goterms".
			goterms=$(echo "$line" | cut -f"$begin_goterms"-"$max_field")
			
			# Assign values in the variable "goterms" to a new indexed array (called "array"), 
			# with tab delimiter (IFS=$'\t')
			IFS=$'\t' read -r -a array <<<"$goterms"
			
			# Iterate through each element of the array.
			# Print the first 12 fields (i.e. the fields stored in "fixed_fields") followed by a tab (%s\t).
			# Print the current element in the array (i.e. the current GO term) followed by a new line (%s\n).
			for element in "${!array[@]}"	
				do printf "%s\t%s\n" "$fixed_fields" "${array[$element]}"
			done
	fi

# Send the input file into the while loop and send the output to a file named "rhonda_fixed.txt".
done < "$file" > "$output_file"

In [76]:
!tail _blast-GO-unfolded.tab

X5M5N0	NC_035787.1:500675-520175	GO:0000122
X5M5N0	NC_035787.1:500675-520175	GO:0004674
X5M5N0	NC_035787.1:500675-520175	GO:0005524
X5M5N0	NC_035787.1:500675-520175	GO:0005829
X5M5N0	NC_035787.1:500675-520175	GO:0006468
X5M5N0	NC_035787.1:500675-520175	GO:0006972
X5M5N0	NC_035787.1:500675-520175	GO:0007275
X5M5N0	NC_035787.1:500675-520175	GO:0035556
X5M5N0	NC_035787.1:500675-520175	GO:0050891
X5M5N0	NC_035787.1:500675-520175	GO:0061404


In [82]:
!grep "GO:0051443" _blast-GO-unfolded.tab

A0A0B4J1F4	NC_035782.1:29450394-29476971	GO:0051443
A0A0B4J1F4	NC_035782.1:29464765-29467517	GO:0051443
Q2TA25	NC_035781.1:12135742-12154474	GO:0051443
Q32KX7	NC_035788.1:79138936-79159683	GO:0051443
Q4V8B2	NC_035781.1:6856575-6864932	GO:0051443
Q5ZKU1	NC_035784.1:71589964-71595119	GO:0051443
Q80Z37	NC_035781.1:32627408-32631993	GO:0051443
Q80Z37	NC_035781.1:46700681-46705505	GO:0051443
Q8CCA0	NC_035782.1:49696486-49708535	GO:0051443
Q969H0	NC_035782.1:65318020-65326555	GO:0051443
Q9UNE7	NC_035781.1:11182440-11192882	GO:0051443


In [58]:
!sort -k3 _blast-GO-unfolded.tab > _blast-GO-unfolded.sorted

In [98]:
!head _blast-GO-unfolded.sorted

A0AUP1	NC_035783.1:30788547-30795846
A0JM56	NC_035783.1:16575143-16593937
A0JNW5	NC_035780.1:63199974-63216977
A0JP70	NC_035781.1:1774563-1776214
A0JP70	NC_035781.1:2272618-2290963
A0JPG1	NC_035784.1:96086824-96130827
A0JPI9	NC_035783.1:45735603-45761596
A0JPI9	NC_035784.1:46205705-46241787
A0JPQ7	NC_035783.1:42618142-42630215
A1A5Q7	NC_035781.1:45122264-45137524


In [87]:
!grep "GO:0051443" _blast-GO-unfolded.sorted

A0A0B4J1F4	NC_035782.1:29450394-29476971	GO:0051443
A0A0B4J1F4	NC_035782.1:29464765-29467517	GO:0051443
Q2TA25	NC_035781.1:12135742-12154474	GO:0051443
Q32KX7	NC_035788.1:79138936-79159683	GO:0051443
Q4V8B2	NC_035781.1:6856575-6864932	GO:0051443
Q5ZKU1	NC_035784.1:71589964-71595119	GO:0051443
Q80Z37	NC_035781.1:32627408-32631993	GO:0051443
Q80Z37	NC_035781.1:46700681-46705505	GO:0051443
Q8CCA0	NC_035782.1:49696486-49708535	GO:0051443
Q969H0	NC_035782.1:65318020-65326555	GO:0051443
Q9UNE7	NC_035781.1:11182440-11192882	GO:0051443


In [97]:
!grep "GO:" _blast-GO-unfolded.sorted | head

P19971	NC_035786.1:35005823-35053811	GO:0000002
P38766	NC_035784.1:81157412-81161455	GO:0000002
P38766	NC_035785.1:6920706-6928297	GO:0000002
P38766	NC_035788.1:25841487-25842908	GO:0000002
Q1HG60	NC_035788.1:84032938-84038358	GO:0000002
Q1HG60	NC_035788.1:84034120-84038279	GO:0000002
Q38CE9	NC_035788.1:65057880-65082026	GO:0000002
Q5AXT5	NC_035781.1:55490481-55502256	GO:0000002
Q5AXT5	NC_035782.1:1147125-1157055	GO:0000002
Q5AXT5	NC_035784.1:87478898-87496133	GO:0000002


In [99]:
!grep "GO:" _blast-GO-unfolded.sorted > _blast-GO-unfolded.sorted2

In [79]:
!head GO-GOslim.sorted

GO:0000001	mitochondrion inheritance	cell organization and biogenesis	P
GO:0000002	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000003	reproduction	other biological processes	P
GO:0000006	high affinity zinc uptake transmembrane transporter activity	transporter activity	F
GO:0000007	low-affinity zinc ion transmembrane transporter activity	transporter activity	F
GO:0000009	"alpha-1,6-mannosyltransferase activity"	other molecular function	F
GO:0000010	trans-hexaprenyltranstransferase activity	other molecular function	F
GO:0000011	vacuole inheritance	cell organization and biogenesis	P
GO:0000012	single strand break repair	DNA metabolism	P
GO:0000012	single strand break repair	stress response	P


In [96]:
!awk -F '\t' '{print NF}' _blast-GO-unfolded.sorted | tail

3
3
3
3
3
3
3
3
3
3


In [85]:
!grep "GO:0051443" GO-GOslim.sorted

GO:0051443	positive regulation of ubiquitin-protein ligase activity	other biological processes	P


In [106]:
#joining files to get GOslim for each query (with duplicate GOslim / query removed)
!join -1 3 -2 1 -t $'\t' \
_blast-GO-unfolded.sorted2 \
GO-GOslim.sorted \
| uniq | awk -F'\t' -v OFS='\t' '{print $3, $1, $5, $6}' \
> Blastquery-GOslim.tab

In [107]:
!head Blastquery-GOslim.tab

NC_035786.1:35005823-35053811	GO:0000002	cell organization and biogenesis	P
NC_035784.1:81157412-81161455	GO:0000002	cell organization and biogenesis	P
NC_035785.1:6920706-6928297	GO:0000002	cell organization and biogenesis	P
NC_035788.1:25841487-25842908	GO:0000002	cell organization and biogenesis	P
NC_035788.1:84032938-84038358	GO:0000002	cell organization and biogenesis	P
NC_035788.1:84034120-84038279	GO:0000002	cell organization and biogenesis	P
NC_035788.1:65057880-65082026	GO:0000002	cell organization and biogenesis	P
NC_035781.1:55490481-55502256	GO:0000002	cell organization and biogenesis	P
NC_035782.1:1147125-1157055	GO:0000002	cell organization and biogenesis	P
NC_035784.1:87478898-87496133	GO:0000002	cell organization and biogenesis	P
